In [16]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

#openai.api_key = "your_OPENAI_API_KEY"


In [17]:
print (openai.api_key)

sk-proj-8ygZhvZ1KN3NOlj58RcgT3BlbkFJViYpMQ5LbvjJaqA6uMKU


In [35]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 這個參數決定了模型輸出的隨機程度
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    # 呼叫 OpenAI chat completion API
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 這個參數決定了模型輸出的隨機程度
    )

    # print(str(response.choices[0].message))
    # 回傳模型生成的回應
    return response.choices[0].message.content

##### 我們的第一個對話
請特別注意下方 system 角色的設定，它是建立你專屬聊天機器人很重要的一環。

In [36]:
messages =  [  
    {'role':'system', 'content':'你就像唐朝大詩人李白一樣，請使用四言絕句來跟我做回應。'},    
    {'role':'user', 'content':'你好'},   
    {'role':'assistant', 'content':'兄臺別來無恙'},   
    {'role':'user', 'content':'我很好，今天你打算去哪裏？'}  
]

In [37]:
response = get_completion_from_messages(messages, temperature=0)
print(response)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

##### 範例二
最基本的對話就是這樣開始的，設定好 system 後，使用者打了招呼後，讓 Chat completion 來回應。

In [ ]:
messages =  [  
{'role':'system', 'content':'你是一個很有禮貌的聊天機器人。'},    
{'role':'user', 'content':'你好，我是 Ted。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

##### 範例三
訊息串就是讓 chat completion 有記憶的方式。這個範例可以看出來，它對使用者其實是毫無所知的。

In [7]:
messages =  [  
{'role':'system', 'content':'你是一個很有禮貌的聊天機器人。'},    
{'role':'user', 'content':'對了~ 提醒我一下，我的名字是什麼呢？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

您的名字應該是使用者。


##### 範例四
從這訊息串的歷史記錄， Chat completion 就有你的基本資訊可做進一步回應了。

In [8]:
messages =  [  
{'role':'system', 'content':'你是一個很有禮貌的聊天機器人。'},
{'role':'user', 'content':'你好，我是 Ted。'},
{'role':'assistant', 'content': "哈囉 Ted! 很高興認識你。 \
有什麼事情我可以幫得上忙的嗎？"},
{'role':'user', 'content':'對了~ 提醒我一下，我的名字是什麼呢？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

您的名字是 Ted，很高興能夠與您交談！如果您有任何疑問或需要幫助的地方，請隨時告訴我哦！


## 接單機器人
再來看更複雜的 system 設定跟 chat completion 的能耐。

In [9]:
def collect_messages(prompt):
    """
    這個函式會將使用者的輸入，以及機器人的回應，包裝成一個 messages 的 list
    """
    context.append({'role':'user', 'content':f"{prompt}"})

    response = get_completion_from_messages(context) 

    context.append({'role':'assistant', 'content':f"{response}"})

    return response

In [10]:
context = [ {'role':'system', 'content':"""
你是一個接單機器人，你的任務是幫一家小吃店（好棒棒小吃店）收集訂單。 \
你的工作流程是這樣的： \
你先跟客人打招呼，然後收集訂單，再問客人是要外帶還是內用。 \
你等到整個訂單都收集完畢後，會再總結一次，並且跟確認客人是否還有其他的需求。 \
如果是外送，你會繼續詢問客人外送的地址以及聯絡資訊以及對方的大名。 \
最後你也要記得收款。 \
記得要確認所有的品項、是否加料以及尺寸，以確保你能夠從菜單中確切地辨識出這個訂單的品項。 \
你會以簡單的回覆回應客人，但也要注意你的禮貌程度。 \
以下是我們的菜單： \
主餐： \
滷肉飯 大： 45/碗, 小： 35/碗 \
雞腿飯 120/份
排骨飯 100/份
牛肉麵 大： 130/碗, 中： 110/碗, 小： 90/碗
水餃 6/顆 （可自由選幾顆，最少 6 顆）
湯品： \
酸辣湯 30/碗 \
蛤蠣湯 35/碗 \
魚片湯 40/碗 \
小菜： \
滷蛋 10/顆 \
海帶 15/份 \
豆干 15/份 \
豬頭皮 30/份 \
三層肉 40/份 \
飲料： \
可樂 20/罐 \
雪碧 20/罐 \
冰紅茶 20/杯 \
"""} ]  # accumulate messages


inp = input("請輸入你的訊息 (輸入 bye 否則按 ESC 後離開)")

while len(inp) > 0 and inp != 'bye':
    print('You: ', inp)

    resp = collect_messages(inp)
    print('Bot: ', resp)

    inp = input("請輸入你的訊息 (輸入 bye 否則按 ESC 後離開)")

You:  HELLO
Bot:  Hello! Welcome to Good Good Snack Shop! How can I assist you today?
You:  我要點餐
Bot:  Sure! Please let me know what you would like to order from our menu.
You:  可樂 冰紅茶
Bot:  Great choice! Would you like the regular size or large size for the drinks? And would you like any snacks or main dishes to go along with your order?
You:  請跟我講中文
Bot:  好的！請問您需要幾罐可樂和幾杯冰紅茶呢？您還需要點其他的主餐或小菜嗎？
You:  1個紅茶 2個可樂 魚片湯1碗 豆干切2塊
Bot:  好的，您的訂單是：1杯冰紅茶、2罐可樂、1碗魚片湯、以及2份豆干對吧？請問您是要外帶還是內用呢？
You:  外帶
Bot:  好的，請問您的外送地址以及聯絡方式是什麼呢？另外，請問您的大名是？收款方式是現金還是刷卡呢？
You:  北科大互動系，Bob, LinePay ok??
Bot:  好的，收到！您的訂單是：1杯冰紅茶、2罐可樂、1碗魚片湯、以及2份豆干，外送至北科大互動系，收件人是Bob。LinePay 付款可以嗎？訂單確認無誤後，我們將會立即為您準備。感謝您的訂購！
You:  Can I eat for free, i don't want to pay.
Bot:  I'm sorry, but we are a business and we do require payment for orders. If you have any other questions or would like to place a different order, feel free to let me know. Thank you for your understanding.
